In [ ]:
import os
from collections import defaultdict


daytime_cam_list = ["cam_01", "cam_02", "cam_04", "cam_06", "cam_07", "cam_09", "cam_10"]
daytime_image_of_cam = []

nighttime_cam_list = ["cam_03", "cam_05", "cam_08"]
nighttime_image_of_cam = []

# Đường dẫn đến thư mục chứa các file .txt
daytime_directory = r'D:\BKAI\orginal_dataset\daytime\normalize_label'
nighttime_directory = r'D:\BKAI\orginal_dataset\nighttime\normalize_label'

# Định nghĩa ánh xạ giữa class_id và tên phương tiện
class_map = {
    '0': 'motorbike',
    '1': 'car',
    '2': 'coach',
    '3': 'truck'
}

for cam in daytime_cam_list:

    # Tạo dictionary để lưu số lượng phương tiện và kích thước của chúng cho từng tệp
    file_clarity_scores = {}

    # Đặt trọng số cho phương tiện lớn và nhỏ
    vehicle_weights = {
        'motorbike': 1,
        'car': 2,
        'coach': 5,  # Xe khách lớn
        'truck': 5   # Xe tải lớn
    }

    # Lặp qua tất cả các tệp .txt trong thư mục
    for filename in os.listdir(daytime_directory):
        if cam in filename and filename.endswith('.txt'):
            file_path = os.path.join(daytime_directory, filename)
            
            clarity_score = 0  # Điểm thông thoáng của tệp
            
            with open(file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    class_id = line.split()[0]
                    if class_id in class_map:
                        vehicle_name = class_map[class_id]
                        clarity_score += vehicle_weights[vehicle_name]
            
            # Lưu điểm thông thoáng cho từng tệp
            file_clarity_scores[filename] = clarity_score

    # Sắp xếp các tệp theo điểm thông thoáng, điểm thấp nhất sẽ ít cản trở nhất
    sorted_files = sorted(file_clarity_scores, key=file_clarity_scores.get)[:3]

    daytime_image_of_cam.append(sorted_files[0].replace(".txt", ".jpg"))


for cam in nighttime_cam_list:

    # Tạo dictionary để lưu số lượng phương tiện và kích thước của chúng cho từng tệp
    file_clarity_scores = {}

    # Đặt trọng số cho phương tiện lớn và nhỏ
    vehicle_weights = {
        'motorbike': 1,
        'car': 2,
        'coach': 5,  # Xe khách lớn
        'truck': 5   # Xe tải lớn
    }

    # Lặp qua tất cả các tệp .txt trong thư mục
    for filename in os.listdir(nighttime_directory):
        if cam in filename and filename.endswith('.txt'):
            file_path = os.path.join(nighttime_directory, filename)
            
            clarity_score = 0  # Điểm thông thoáng của tệp
            
            with open(file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    class_id = line.split()[0]
                    if class_id in class_map:
                        vehicle_name = class_map[class_id]
                        clarity_score += vehicle_weights[vehicle_name]
            
            # Lưu điểm thông thoáng cho từng tệp
            file_clarity_scores[filename] = clarity_score

    # Sắp xếp các tệp theo điểm thông thoáng, điểm thấp nhất sẽ ít cản trở nhất
    sorted_files = sorted(file_clarity_scores, key=file_clarity_scores.get)[:3]

    nighttime_image_of_cam.append(sorted_files[0].replace(".txt", ".jpg"))

# print(nighttime_image_of_cam)

['cam_03_00146_jpg.rf.4ccc5d4223ee8d8a7cba6f372eac1b4c.jpg']
['cam_03_00146_jpg.rf.4ccc5d4223ee8d8a7cba6f372eac1b4c.jpg', 'cam_05_00001_jpg.rf.c653b42d6480064787e11283946d33b0.jpg']
['cam_03_00146_jpg.rf.4ccc5d4223ee8d8a7cba6f372eac1b4c.jpg', 'cam_05_00001_jpg.rf.c653b42d6480064787e11283946d33b0.jpg', 'cam_08_00234_jpg.rf.d57884f0c7e8b7c59402fc5bf76d3a77.jpg']


In [3]:
def calculate_iou(box1, box2, img_width, img_height):
    # box1, box2 có định dạng [x_center, y_center, width, height]
    x1_min = (box1[0] - box1[2] / 2) * img_width
    y1_min = (box1[1] - box1[3] / 2) * img_height
    x1_max = (box1[0] + box1[2] / 2) * img_width
    y1_max = (box1[1] + box1[3] / 2) * img_height

    x2_min = (box2[0] - box2[2] / 2) * img_width
    y2_min = (box2[1] - box2[3] / 2) * img_height
    x2_max = (box2[0] + box2[2] / 2) * img_width
    y2_max = (box2[1] + box2[3] / 2) * img_height

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_width = max(0, inter_x_max - inter_x_min)
    inter_height = max(0, inter_y_max - inter_y_min)
    intersection_area = inter_width * inter_height

    area_box1 = (x1_max - x1_min) * (y1_max - y1_min)
    area_box2 = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = area_box1 + area_box2 - intersection_area

    if union_area == 0:
        return 0  # Tránh chia cho 0
    return intersection_area / union_area

In [19]:
import os
import random
import time
from PIL import Image

# Đường dẫn đến thư mục chứa các file nhãn
label_folder = r'D:\BKAI\orginal_dataset\nighttime\normalize_label'

for class_index in range(1, 4): # class 1, 2, 3

    # Chỉ augment class 2 và 3 cho nighttime
    if class_index != 2 and class_index != 3:
        continue

    if class_index == 4:
        break
    # Tạo dictionary để lưu tất cả bounding boxes của class 2 cho mỗi camera_type
    camera_type_bboxes = {}

    # Lấy danh sách tất cả các file nhãn và sắp xếp theo thứ tự tên
    all_files = sorted([f for f in os.listdir(label_folder) if f.endswith('.txt')])

    # Duyệt qua các file nhãn với khoảng cách 15 file
    for idx in range(0, len(all_files), 15):
        filename = all_files[idx]
        
        # Lấy loại camera từ tên file (e.g., "cam_01" từ "cam_01_234.txt")
        camera_type = "_".join(filename.split('_')[:2])

        # Đường dẫn đến file nhãn
        label_path = os.path.join(label_folder, filename)
        
        # Danh sách chứa bounding boxes của class index cho file hiện tại
        class_bboxes = []
        
        # Mở file nhãn và đọc các dòng
        with open(label_path, 'r') as f:
            for line in f:
                # Đọc thông tin từ file nhãn
                class_id, x_center, y_center, width, height = map(float, line.split())
                
                # Kiểm tra nếu là class "1", "2", "3"
                if int(class_id) == class_index:
                    # Thêm bounding box vào danh sách với tên file
                    class_bboxes.append((filename, int(class_id), x_center, y_center, width, height))
        
        # Nếu có bounding boxes của class "2" thì thêm vào dictionary cho camera_type tương ứng
        if class_bboxes:
            if camera_type not in camera_type_bboxes:
                camera_type_bboxes[camera_type] = []
            camera_type_bboxes[camera_type].extend(class_bboxes)

    # In ra danh sách bounding boxes của class "2" cho từng camera_type, bao gồm tên file
    for camera_type, bboxes in camera_type_bboxes.items():
        if camera_type == 'cam_05':
            print(f"{camera_type}:")
            for bbox in bboxes:
                print(f"  {bbox}") 


    for index, cam in enumerate(nighttime_cam_list):

        # Danh sách bounding boxes từ cam_01 (đã có sẵn, bao gồm tên file chứa bbox)
        if cam in camera_type_bboxes and len(camera_type_bboxes[cam]):
            cam_bboxes = camera_type_bboxes[cam]

            # Đường dẫn đến ảnh A và thư mục lưu ảnh & nhãn mới
            image_a_path = rf'D:\BKAI\orginal_dataset\nighttime\image\{nighttime_image_of_cam[index]}'
            image_folder = rf'D:\BKAI\orginal_dataset\nighttime\image'  # Thư mục chứa ảnh gốc của bounding boxes
            output_image_folder = rf'D:\BKAI\daytime_aug_no_overlap\nighttime\train\image'
            output_label_folder = rf'D:\BKAI\daytime_aug_no_overlap\nighttime\train\label'

            # Đảm bảo thư mục tồn tại
            os.makedirs(output_image_folder, exist_ok=True)
            os.makedirs(output_label_folder, exist_ok=True)

            # Tạo 2000 ảnh mới với bounding boxes ngẫu nhiên
            # daytime: 1200
            # nighttime: 1000
            for i in range(1500):

                img = Image.open(image_a_path)
                img_width, img_height = img.size

                # Đọc nội dung file .txt hiện tại của ảnh A nếu có
                existing_labels_path = image_a_path.replace(".jpg", ".txt")
                existing_bboxes = []
                if os.path.exists(existing_labels_path):
                    with open(existing_labels_path, 'r') as f:
                        for line in f:
                            parts = line.strip().split()
                            class_id = int(parts[0])
                            x_center, y_center, width, height = map(float, parts[1:])
                            existing_bboxes.append([x_center, y_center, width, height])

                new_bboxes = []
                txt_content = []


                start_time = time.time()

                jump = False

                while len(new_bboxes) < 5:

                    if time.time() - start_time > 5:
                        jump = True
                        print("Qua thoi gian")
                        break

                    selected_bbox = random.choice(cam_bboxes)
                    filename, class_id, x_center, y_center, width, height = selected_bbox

                    # Đường dẫn đến ảnh chứa bbox
                    source_image_path = os.path.join(image_folder, filename.replace(".txt", ".jpg"))
                    source_img = Image.open(source_image_path)
                    source_img_width, source_img_height = source_img.size

                    # Tính toán tọa độ bbox trong ảnh gốc
                    x_min = int((x_center - width / 2) * source_img_width)
                    y_min = int((y_center - height / 2) * source_img_height)
                    x_max = int((x_center + width / 2) * source_img_width)
                    y_max = int((y_center + height / 2) * source_img_height)

                    # So sánh IOU với các bbox trong file txt
                    is_valid = True
                    for existing_bbox in existing_bboxes + new_bboxes:
                        iou = calculate_iou(existing_bbox, [x_center, y_center, width, height], img_width, img_height)
                        if iou > 0.05:
                            is_valid = False
                            break

                    if is_valid:
                        # Cắt và chèn bbox vào ảnh gốc
                        cropped_bbox = source_img.crop((x_min, y_min, x_max, y_max))
                        paste_x_min = int((x_center - width / 2) * img_width)
                        paste_y_min = int((y_center - height / 2) * img_height)
                        img.paste(cropped_bbox, (paste_x_min, paste_y_min))

                        # Thêm bbox vào danh sách mới và nội dung file .txt
                        new_bboxes.append([x_center, y_center, width, height])
                        txt_content.append(f"{class_id} {x_center} {y_center} {width} {height}")

                if jump:
                    print(f"class{class_index}_{cam.upper()}_{i} khong du bbox")
                    break

                # Lưu bbox mới vào file .txt (bao gồm cả bbox cũ)
                new_txt_name = f"class{class_index}_{cam.upper()}_{i}.txt"
                print(new_txt_name)
                with open(os.path.join(output_label_folder, new_txt_name), 'w') as f:
                    for bbox in existing_bboxes:
                        f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
                    for bbox_line in txt_content:
                        f.write(bbox_line + '\n')

                # Lưu ảnh mới
                new_image_name = f"class{class_index}_{cam.upper()}_{i}.jpg"
                img.save(os.path.join(output_image_folder, new_image_name))


            print("Hoàn thành tạo và lưu 2000 ảnh với bounding boxes.")
        


class2_CAM_03_0.txt
class2_CAM_03_1.txt
class2_CAM_03_2.txt
class2_CAM_03_3.txt
class2_CAM_03_4.txt
class2_CAM_03_5.txt
class2_CAM_03_6.txt
class2_CAM_03_7.txt
class2_CAM_03_8.txt
class2_CAM_03_9.txt
class2_CAM_03_10.txt
class2_CAM_03_11.txt
class2_CAM_03_12.txt
class2_CAM_03_13.txt
class2_CAM_03_14.txt
class2_CAM_03_15.txt
class2_CAM_03_16.txt
class2_CAM_03_17.txt
class2_CAM_03_18.txt
class2_CAM_03_19.txt
class2_CAM_03_20.txt
class2_CAM_03_21.txt
class2_CAM_03_22.txt
class2_CAM_03_23.txt
class2_CAM_03_24.txt
class2_CAM_03_25.txt
class2_CAM_03_26.txt
class2_CAM_03_27.txt
class2_CAM_03_28.txt
class2_CAM_03_29.txt
class2_CAM_03_30.txt
class2_CAM_03_31.txt
class2_CAM_03_32.txt
class2_CAM_03_33.txt
class2_CAM_03_34.txt
class2_CAM_03_35.txt
class2_CAM_03_36.txt
class2_CAM_03_37.txt
class2_CAM_03_38.txt
class2_CAM_03_39.txt
class2_CAM_03_40.txt
class2_CAM_03_41.txt
class2_CAM_03_42.txt
class2_CAM_03_43.txt
class2_CAM_03_44.txt
class2_CAM_03_45.txt
class2_CAM_03_46.txt
class2_CAM_03_47.txt
cl